# SQL Subqueries - Lab Assignment #2

## Introduction

Now that you've seen how subqueries work, it's time to get some practice writing them! Not all of the queries will require subqueries, but all will be a bit more complex and require some thought and review about aggregates, grouping, ordering, filtering, joins and subqueries. Good luck!  

## Objectives

You will be able to:

* Write subqueries to decompose complex queries

## CRM Database ERD

Once again, here's the schema for the CRM database you'll continue to practice with.

<img src="img/Database-Schema.png" width="600">

## Connect to the Database

As usual, start by importing the necessary packages and connecting to the database `data2.sqlite` in the data folder.

In [1]:
# Your code here; import the necessary packages
import sqlite3
import pandas as pd

In [2]:
# Your code here; create the connection

conn = sqlite3.Connection("data/data2.sqlite")

## Write an Equivalent Query using a Subquery

The following query works using a `JOIN`. Rewrite it so that it uses a subquery instead.

```
SELECT
    customerNumber,
    contactLastName,
    contactFirstName
FROM customers
JOIN orders 
    USING(customerNumber)
WHERE orderDate = '2003-01-31'
;
```

In [3]:
# Your code here
pd.read_sql('''
SELECT 
    customerNumber
    ,contactLastName
    ,contactFirstName
FROM (
    SELECT
        *
    FROM customers
    JOIN orders 
        USING(customerNumber)
    WHERE orderDate = '2003-01-31')
''', conn)

,customerNumber,contactLastName,contactFirstName
0,141,Freyre,Diego


## Select the Total Number of Orders for Each Product Name

Sort the results by the total number of items sold for that product.

In [4]:
# Your code here
pd.read_sql('''
    SELECT
        products.productName
        ,orderdetails.productCode
        ,SUM(orderdetails.quantityOrdered) as total_number_of_orders
    FROM orderdetails
        JOIN orders 
            ON orderdetails.orderNumber = orders.orderNumber
        JOIN products
            ON orderdetails.productCode = products.productCode
    GROUP BY
        products.productName  
''', conn)

,productName,productCode,total_number_of_orders
0,18th Century Vintage Horse Carriage,S18_3136,907
1,18th century schooner,S24_2011,1011
2,1900s Vintage Bi-Plane,S24_2841,940
3,1900s Vintage Tri-Plane,S24_4278,1009
4,1903 Ford Model A,S18_3140,883
...,...,...,...
104,The Mayflower,S700_1938,898
105,The Queen Mary,S700_3962,896
106,The Schooner Bluenose,S700_1138,934
107,The Titanic,S700_3505,952


## Select the Product Name and the  Total Number of People Who Have Ordered Each Product

Sort the results in descending order.

### A quick note on the SQL  `SELECT DISTINCT` statement:

The `SELECT DISTINCT` statement is used to return only distinct values in the specified column. In other words, it removes the duplicate values in the column from the result set.

Inside a table, a column often contains many duplicate values; and sometimes you only want to list the unique values. If you apply the `DISTINCT` clause to a column that has `NULL`, the `DISTINCT` clause will keep only one NULL and eliminates the other. In other words, the DISTINCT clause treats all `NULL` “values” as the same value.

In [5]:
# Your code here
# Hint: because one of the tables we'll be joining has duplicate customer numbers, you should use DISTINCT

# Your code here
pd.read_sql('''
  SELECT
        products.productCode
        ,COUNT(DISTINCT customers.customerNumber) AS Total_Customers
    FROM products
        JOIN orderdetails 
            ON products.productCode = orderdetails.productCode
        JOIN orders
            ON orderdetails.orderNumber = orders.orderNumber
        JOIN customers
            ON orders.customerNumber = customers.customerNumber
    GROUP BY products.productCode
    ORDER BY COUNT(DISTINCT customers.customerNumber) DESC
''', conn)

,productCode,Total_Customers
0,S18_3232,40
1,S72_1253,27
2,S18_2957,27
3,S10_4757,27
4,S10_1949,27
...,...,...
104,S24_2840,19
105,S24_3432,18
106,S24_3191,18
107,S24_2887,18


## Select the Employee Number, First Name, Last Name, City (of the office), and Office Code of the Employees Who Sold Products That Have Been Ordered by Fewer Than 20 people.

This problem is a bit tougher. To start, think about how you might break the problem up. Be sure that your results only list each employee once.

In [6]:
# Your code here
pd.read_sql('''
    SELECT DISTINCT
        employees.employeeNumber
        ,employees.firstName
        ,employees.lastName
        ,employees.officeCode
        ,orderdetails.quantityOrdered
    FROM employees 
        JOIN customers
            ON employees.employeeNumber = customers.salesRepEmployeeNumber
        JOIN orders
            ON customers.customerNumber = orders.customerNumber
        JOIN orderdetails
            ON orders.orderNumber = orderdetails.orderNumber
    WHERE
        orderdetails.quantityOrdered < 20
    GROUP BY
        employeeNumber
    
''', conn)

,employeeNumber,firstName,lastName,officeCode,quantityOrdered
0,1165,Leslie,Jennings,1,6
1,1323,George,Vanauf,3,11
2,1370,Gerard,Hernandez,4,19
3,1401,Pamela,Castillo,4,15
4,1611,Andy,Fixter,6,18
5,1612,Peter,Marsh,6,6
6,1621,Mami,Nishi,5,15


## Select the Employee Number, First Name, Last Name, and Number of Customers for Employees Whose Customers Have an Average Credit Limit Over 15K

In [7]:
# Your code here
pd.read_sql('''
    SELECT
        employees.employeeNumber
        ,employees.firstName
        ,employees.lastName
        ,AVG(customers.creditLimit) AS avgcreditLimit
    FROM employees 
        JOIN customers
            ON employees.employeeNumber = customers.salesRepEmployeeNumber
    GROUP BY
        employeeNumber
    HAVING
        AVG(customers.creditLimit) > 15000
    
''', conn)


,employeeNumber,firstName,lastName,avgcreditLimit
0,1165,Leslie,Jennings,100433.333333
1,1166,Leslie,Thompson,65266.666667
2,1188,Julie,Firrelli,73916.666667
3,1216,Steve,Patterson,81533.333333
4,1286,Foon Yue,Tseng,66614.285714
5,1323,George,Vanauf,80887.500000
6,1337,Loui,Bondur,86233.333333
7,1370,Gerard,Hernandez,91785.714286
8,1401,Pamela,Castillo,81340.000000
9,1501,Larry,Bott,91187.500000


## Summary

In this lesson, you got to practice some more complex SQL queries, some of which required subqueries. There's still plenty more SQL to be had though; hope you've been enjoying some of these puzzles!